<a href="https://colab.research.google.com/github/Ponlibarnaa/Dataset/blob/main/3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from pandas import DataFrame 
df_buy  = pd.read_csv("https://raw.githubusercontent.com/Ponlibarnaa/Dataset2/main/Data_Sheet1.csv")
print("\n Given Buying Systems Data Set:\n\n", df_buy)




 Given Buying Systems Data Set:

     RID     AGE  INCOME STUDENT CREDIT SALARY LABEL
0     1   Youth    High      No          Fair    No
1     2   Youth    High      No     Excellent    No
2     3  Middle    High      No          Fair   Yes
3     4  Senior  Middle      No          Fair   Yes
4     5  Senior     Low     Yes          Fair   Yes
5     6  Senior  Middle     Yes     Excellent    No
6     7   Youth  Middle      No          Fair    No
7     8  Middle  Middle     Yes          Fair   Yes
8     9   Youth  Middle     Yes          Fair   Yes
9    10  Senior  Middle     Yes          Fair   Yes
10   11   Youth  Middle     Yes     Excellent   Yes
11   12  Middle  Middle      No          Fair   Yes
12   13  Middle    High     Yes          Fair   Yes
13   14  Senior  Middle      No     Excellent    No


In [7]:
from collections import Counter
import math
def entropy(probs):  
    return sum( [-prob*math.log(prob, 2) for prob in probs] )

def entropyCalc(a_list):  
    c = Counter(a_list) 
    num_instances = len(a_list)*1.0  
    print("\nNumber of Instances is {0}.".format(num_instances))
    probs = [x / num_instances for x in c.values()]  
    print("\nThe Classes are:",min(c),"and",max(c))
    print("Probabilities of Class {0} is {1}.".format(min(c),min(probs)))
    print("Probabilities of Class {0} is {1}.".format(max(c),max(probs)))
    return entropy(probs) 
    

print("\nInput sample for entropy calculation is:", df_buy['LABEL'])
total_entropy = entropyCalc(df_buy['LABEL'])
print("\nTotal Entropy of Buying Systems Data Set is:",total_entropy)


Input sample for entropy calculation is: 0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6      No
7     Yes
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     No
Name: LABEL, dtype: object

Number of Instances is 14.0.

The Classes are: No and Yes
Probabilities of Class No is 0.35714285714285715.
Probabilities of Class Yes is 0.6428571428571429.

Total Entropy of Buying Systems Data Set is: 0.9402859586706309


In [9]:


def information_gain(df, split_attribute_name, target_attribute_name, trace=0):
    print("Information Gain Calculation of ",split_attribute_name)
    df_split = df.groupby(split_attribute_name)
    nobs = len(df.index) * 1.0
    df_agg_ent = df_split.agg({target_attribute_name : [entropyCalc, lambda x: len(x)/nobs] })[target_attribute_name]
    df_agg_ent.columns = ['Entropy', 'PropObservations']

    new_entropy = sum( df_agg_ent['Entropy'] * df_agg_ent['PropObservations'] )
    old_entropy = entropyCalc(df[target_attribute_name])
    return old_entropy - new_entropy

print('Info-gain for Age is :'+str( information_gain(df_buy, 'AGE', 'LABEL')),"\n")
print('\n Info-gain for Income is: ' + str( information_gain(df_buy, 'INCOME', 'LABEL')),"\n")
print('\n Info-gain for Credit is:' + str( information_gain(df_buy, 'CREDIT SALARY', 'LABEL')),"\n")




Information Gain Calculation of  AGE

Number of Instances is 4.0.

The Classes are: Yes and Yes
Probabilities of Class Yes is 1.0.
Probabilities of Class Yes is 1.0.

Number of Instances is 5.0.

The Classes are: No and Yes
Probabilities of Class No is 0.4.
Probabilities of Class Yes is 0.6.

Number of Instances is 5.0.

The Classes are: No and Yes
Probabilities of Class No is 0.4.
Probabilities of Class Yes is 0.6.

Number of Instances is 14.0.

The Classes are: No and Yes
Probabilities of Class No is 0.35714285714285715.
Probabilities of Class Yes is 0.6428571428571429.
Info-gain for Age is :0.2467498197744391 

Information Gain Calculation of  INCOME

Number of Instances is 4.0.

The Classes are: No and Yes
Probabilities of Class No is 0.5.
Probabilities of Class Yes is 0.5.

Number of Instances is 1.0.

The Classes are: Yes and Yes
Probabilities of Class Yes is 1.0.
Probabilities of Class Yes is 1.0.

Number of Instances is 9.0.

The Classes are: No and Yes
Probabilities of Class N

In [14]:


def id3(df, target_attribute_name, attribute_names, default_class=None):
    from collections import Counter
    c = Counter(x for x in df[target_attribute_name])
    if len(c) == 1:
        return next(iter(c))
    elif df.empty or (not attribute_names):
        return default_class 
    else:
        default_class = max(c.keys()) 
        gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_names] 
        index_of_max = gainz.index(max(gainz)) 
        best_attr = attribute_names[index_of_max]
        tree = {best_attr:{}}  
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset,
                        target_attribute_name,
                        remaining_attribute_names,
                        default_class)
            tree[best_attr][attr_val] = subtree
        return tree


In [15]:
attribute_names = list(df_buy.columns)
print("List of Attributes in Sample:", attribute_names) 
attribute_names.remove('LABEL') 
attribute_names.remove('RID') 
print("Predicting these Attributes:", attribute_names)


List of Attributes in Sample: ['RID', 'AGE', 'INCOME', 'STUDENT', 'CREDIT SALARY', 'LABEL', 'predicted']
Predicting these Attributes: ['AGE', 'INCOME', 'STUDENT', 'CREDIT SALARY', 'predicted']


In [16]:
from pprint import pprint
tree = id3(df_buy,'LABEL',attribute_names)
print("\n\nThe Completed Decision Tree is :\n")

pprint(tree)
attribute = next(iter(tree))
print("Best Attribute :\n",attribute)
print("Tree Keys:\n",tree[attribute].keys())

Information Gain Calculation of  AGE

Number of Instances is 4.0.

The Classes are: Yes and Yes
Probabilities of Class Yes is 1.0.
Probabilities of Class Yes is 1.0.

Number of Instances is 5.0.

The Classes are: No and Yes
Probabilities of Class No is 0.4.
Probabilities of Class Yes is 0.6.

Number of Instances is 5.0.

The Classes are: No and Yes
Probabilities of Class No is 0.4.
Probabilities of Class Yes is 0.6.

Number of Instances is 14.0.

The Classes are: No and Yes
Probabilities of Class No is 0.35714285714285715.
Probabilities of Class Yes is 0.6428571428571429.
Information Gain Calculation of  INCOME

Number of Instances is 4.0.

The Classes are: No and Yes
Probabilities of Class No is 0.5.
Probabilities of Class Yes is 0.5.

Number of Instances is 1.0.

The Classes are: Yes and Yes
Probabilities of Class Yes is 1.0.
Probabilities of Class Yes is 1.0.

Number of Instances is 9.0.

The Classes are: No and Yes
Probabilities of Class No is 0.3333333333333333.
Probabilities of C

In [17]:
def classify(instance, tree, default=None): 
    attribute = next(iter(tree))     
    print("Key:",tree.keys())  
    print("Attribute:",attribute)  
    if instance[attribute] in tree[attribute].keys(): 
        result = tree[attribute][instance[attribute]]
        print("Instance Attribute:",instance[attribute],"TreeKeys :",tree[attribute].keys())
        if isinstance(result, dict): 
            return classify(instance, result)
        else:
            return result
    else:
        return default

df_buy['predicted'] = df_buy.apply(classify, axis=1, args=(tree,'No') ) 
print(df_buy['predicted'])
print('\n Accuracy is:\n' + str( sum(df_buy['LABEL']==df_buy['predicted'] ) / (1.0*len(df_buy.index)) ))
df_buy[['LABEL', 'predicted']]

Key: dict_keys(['predicted'])
Attribute: predicted
Instance Attribute: No TreeKeys : dict_keys(['No', 'Yes'])
Key: dict_keys(['predicted'])
Attribute: predicted
Instance Attribute: No TreeKeys : dict_keys(['No', 'Yes'])
Key: dict_keys(['predicted'])
Attribute: predicted
Instance Attribute: Yes TreeKeys : dict_keys(['No', 'Yes'])
Key: dict_keys(['predicted'])
Attribute: predicted
Instance Attribute: Yes TreeKeys : dict_keys(['No', 'Yes'])
Key: dict_keys(['predicted'])
Attribute: predicted
Instance Attribute: Yes TreeKeys : dict_keys(['No', 'Yes'])
Key: dict_keys(['predicted'])
Attribute: predicted
Instance Attribute: No TreeKeys : dict_keys(['No', 'Yes'])
Key: dict_keys(['predicted'])
Attribute: predicted
Instance Attribute: No TreeKeys : dict_keys(['No', 'Yes'])
Key: dict_keys(['predicted'])
Attribute: predicted
Instance Attribute: Yes TreeKeys : dict_keys(['No', 'Yes'])
Key: dict_keys(['predicted'])
Attribute: predicted
Instance Attribute: Yes TreeKeys : dict_keys(['No', 'Yes'])
Key: 

,LABEL,predicted
0,No,No
1,No,No
2,Yes,Yes
3,Yes,Yes
4,Yes,Yes
5,No,No
6,No,No
7,Yes,Yes
8,Yes,Yes
9,Yes,Yes


In [18]:
training_data = df_buy.iloc[1:-4] 
test_data  = df_buy.iloc[-4:] 
train_tree = id3(training_data, 'LABEL', attribute_names)

test_data['predicted2'] = test_data.apply(                            
                                          classify, 
                                          axis=1, 
                                          args=(train_tree,'Yes') )


print ('\n\n Accuracy is : ' + str( sum(test_data['LABEL']==test_data['predicted2'] ) / (1.0*len(test_data.index)) ))

Information Gain Calculation of  AGE

Number of Instances is 2.0.

The Classes are: Yes and Yes
Probabilities of Class Yes is 1.0.
Probabilities of Class Yes is 1.0.

Number of Instances is 4.0.

The Classes are: No and Yes
Probabilities of Class No is 0.25.
Probabilities of Class Yes is 0.75.

Number of Instances is 3.0.

The Classes are: No and Yes
Probabilities of Class No is 0.3333333333333333.
Probabilities of Class Yes is 0.6666666666666666.

Number of Instances is 9.0.

The Classes are: No and Yes
Probabilities of Class No is 0.3333333333333333.
Probabilities of Class Yes is 0.6666666666666666.
Information Gain Calculation of  INCOME

Number of Instances is 2.0.

The Classes are: No and Yes
Probabilities of Class No is 0.5.
Probabilities of Class Yes is 0.5.

Number of Instances is 1.0.

The Classes are: Yes and Yes
Probabilities of Class Yes is 1.0.
Probabilities of Class Yes is 1.0.

Number of Instances is 6.0.

The Classes are: No and Yes
Probabilities of Class No is 0.333333

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
